In [2]:
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import sys
import matplotlib
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import skimage
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [3]:
wikidata = pd.read_json('movies/data/wikidata-movies.json.gz', orient='record', lines=True)

In [4]:
wikidata

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,main_subject,metacritic_id,original_language,publication_date,rotten_tomatoes_id,series,wikidata_id
0,NaN,"[Q228931, Q235384]",Q145,[Q43079418],If I Were You (2012 Canadian film),NaN,[Q859369],tt1587309,If I Were You,NaN,NaN,NaN,Q1860,2012-01-01,m/if_i_were_you_2012,NaN,Q43079072
1,NaN,NaN,Q30,NaN,Krampus: The Devil Returns,NaN,[Q28026639],tt3527772,Krampus: The Devil Returns,NaN,NaN,NaN,Q1860,2016-10-04,m/krampus_the_devil_returns,NaN,Q43302269
2,NaN,"[Q106303, Q467957, Q3345693, Q3340838, Q316419...",Q142,[Q724208],Le Brio,[Q50742],[Q157443],tt6462462,NaN,NaN,NaN,NaN,Q150,2017-01-01,m/le_brio,NaN,Q43400054
3,NaN,NaN,Q30,[Q4718041],A Christmas Prince,NaN,"[Q860626, Q28026639]",tt7608418,A Christmas Prince,NaN,NaN,NaN,Q1860,2017-11-17,m/a_christmas_prince,NaN,Q43401863
4,Q7162696,NaN,Q252,[Q11036795],Pengabdi Setan (2017 film),NaN,[Q200092],tt7076834,Pengabdi Setan,NaN,NaN,NaN,Q9240,2017-01-01,m/satans_slaves,NaN,Q41850450
5,NaN,NaN,Q258,NaN,A Small Town Called Descent,NaN,[Q959790],tt3726330,A Small Town Called Descent,NaN,NaN,NaN,Q1860,2010-07-24,m/a_small_town_called_descent,NaN,Q42048906
6,NaN,"[Q5126010, Q3390414, Q5676024, Q237021]",Q29,[Q51892574],Orbiter 9,NaN,"[Q24925, Q21010853]",tt3469798,Orbiter 9,0.0,NaN,NaN,NaN,2017-04-07,m/orbiter_9,NaN,Q42577704
7,Q18214949,"[Q202381, Q7367121, Q179576, Q20630818, Q28474...",Q30,[Q18608206],The Kindergarten Teacher (2018 film),NaN,[Q130232],tt6952960,The Kindergarten Teacher,NaN,NaN,NaN,Q1860,2018-01-01,m/the_kindergarten_teacher_2018,NaN,Q47461695
8,NaN,NaN,Q30,[Q22073902],Who the Fuck Is That Guy? The Fabulous Journey...,NaN,[Q93204],tt4958390,Who the Fuck is That Guy? The Fabulous Journey...,NaN,NaN,NaN,Q1860,2017-07-21,m/who_the_fk_is_that_guy_the_fabulous_journey_...,NaN,Q47483224
9,NaN,NaN,Q30,NaN,The Black Dove,NaN,[Q130232],tt1844790,The Black Dove,NaN,NaN,NaN,Q1860,2012-04-19,m/the_black_dove,NaN,Q42048757


In [11]:
wikidata[wikidata['made_profit']]

KeyError: '[nan nan nan ... nan nan nan] not in index'

In [24]:
wikidata[wikidata['made_profit'].notnull()]

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,main_subject,metacritic_id,original_language,publication_date,rotten_tomatoes_id,series,wikidata_id
6,NaN,"[Q5126010, Q3390414, Q5676024, Q237021]",Q29,[Q51892574],Orbiter 9,NaN,"[Q24925, Q21010853]",tt3469798,Orbiter 9,0.0,NaN,NaN,NaN,2017-04-07,m/orbiter_9,NaN,Q42577704
69,NaN,NaN,Q30,"[Q3384479, Q351884]",Despicable Me,NaN,[Q157443],tt1323594,Despicable Me,1.0,NaN,movie/despicable-me,Q1860,2010-06-20,m/1214097,NaN,Q4447
73,NaN,"[Q386349, Q1605965, Q3805579, Q271162, Q463226...",Q30,[Q2071],Eraserhead,[Q99],"[Q130232, Q200092, Q5967378]",tt0074486,Eraserhead,1.0,[Q906343],NaN,Q1860,1977-01-01,m/eraserhead,NaN,Q11618
81,Q17017426,"[Q117500, Q1376880, Q11930, Q311169, Q951634, ...",Q30,[Q11930],Dances with Wolves,[Q1558],"[Q130232, Q369747, Q21590660, Q21010853, Q319221]",tt0099348,Dances with Wolves,1.0,NaN,movie/dances-with-wolves,Q1860,1990-11-09,m/dances_with_wolves,NaN,Q20456
84,NaN,"[Q38111, Q211553, Q177311, Q8927, Q173399, Q20...",Q145,[Q25191],Inception,"[Q99, Q387047, Q17, Q90, Q1951, Q7275217, Q126...","[Q496523, Q471839, Q2484376, Q188473, Q319221]",tt1375666,Inception,1.0,"[Q544830, Q4425624]",movie/inception,Q1860,2010-07-08,m/inception,NaN,Q25188
92,NaN,"[Q229313, Q445772, Q727988, Q3163137, Q1372392...",Q16,[Q6385039],Mama (2013 film),"[Q172, Q133116, Q13939]",[Q200092],tt2023587,Mama,1.0,[Q80837],movie/mama,Q1860,2013-01-17,m/mama_2013,NaN,Q29446
113,Q243556,"[Q34012, Q41163, Q95043, Q464714, Q171736, Q32...",Q30,[Q56094],The Godfather,"[Q18438, Q60, Q1408, Q1460]","[Q130232, Q959790, Q7444356, Q21010853, Q52162...",tt0068646,The Godfather,1.0,[Q46952],movie/the-godfather,Q1860,1972-03-15,m/godfather,Q3225260,Q47703
125,NaN,"[Q483118, Q23547, Q108283, Q215072, Q270664, Q...",Q30,[Q483118],Argo (2012 film),"[Q406, Q65, Q43]","[Q622291, Q18620604]",tt1024648,Argo,1.0,"[Q589673, Q1032059, Q23036198]",movie/argo,Q1860,2012-01-01,m/argo_2012,NaN,Q59653
203,Q7857661,"[Q317343, Q57147, Q244674, Q343616, Q208649, Q...",Q145,[Q706475],12 Years a Slave (film),[Q34404],"[Q130232, Q645928, Q52162262]",tt2024544,12 Years a Slave,1.0,"[Q118382, Q8461]",movie/12-years-a-slave,Q1860,2013-08-30,m/12_years_a_slave,NaN,Q3023357
483,NaN,"[Q295803, Q200534, Q228865, Q200405, Q314133, ...",Q145,[Q191755],Only Lovers Left Alive,"[Q183, Q365]","[Q130232, Q1054574, Q2137852]",tt1714915,Only Lovers Left Alive,1.0,[Q46721],movie/only-lovers-left-alive,Q1860,2013-05-25,m/only_lovers_left_alive,NaN,Q3352751


In [17]:
genres = pd.read_json('movies/data/genres.json.gz', orient='record', lines=True)

In [27]:
genres

,genre_label,wikidata_id
0,novella,Q43334491
1,cityscape,Q42304051
2,drama with music,Q39894018
3,lunar art,Q47909472
4,Tsukuri monogatari,Q42922249
5,traditional fairy tale,Q42402646
6,fantasy video game,Q42409239
7,propaganda song,Q42681239
8,miniaturist,Q47494964
9,Breton rock,Q47005557


In [21]:
rottentomatoes = pd.read_json('movies/data/rotten-tomatoes.json.gz', orient='record', lines=True)

In [23]:
rottentomatoes

,audience_average,audience_percent,audience_ratings,critic_average,critic_percent,imdb_id,rotten_tomatoes_id
0,3.9,86.0,33125227.0,7.1,79.0,tt0325980,m/pirates_of_the_caribbean_the_curse_of_the_bl...
1,4.1,80.0,2139.0,7.8,88.0,tt4882376,m/first_they_killed_my_father_a_daughter_of_ca...
2,3.6,71.0,11482.0,7.6,86.0,tt0099763,m/henry_portrait_of_a_serial_killer
3,3.6,66.0,187204.0,6.6,66.0,tt1371111,m/cloud_atlas_2012
4,3.9,84.0,26836.0,7.4,80.0,tt1189073,m/the_skin_i_live_in
5,3.7,74.0,1498.0,7.0,88.0,tt1787725,m/if_a_tree_falls_a_story_of_the_earth_liberat...
6,3.6,85.0,92824.0,6.6,73.0,tt0117665,m/1073595-sleepers
7,3.2,50.0,242347.0,4.9,35.0,tt1611224,m/abraham_lincoln_vampire_hunter
8,4.0,87.0,427.0,7.4,100.0,tt3966544,m/lost_soul_the_doomed_journey_of_richard_stan...
9,3.7,74.0,260776.0,7.4,85.0,tt0780521,m/1196003-princess_and_the_frog


In [25]:
omdb = pd.read_json('movies/data/omdb-data.json.gz', orient='record', lines=True)

In [26]:
omdb

,imdb_id,omdb_awards,omdb_genres,omdb_plot
0,tt0060814,Nominated for 2 Oscars. Another 2 nominations.,"[Drama, History, War]","In this sprawling, star-laden film, we see the..."
1,tt2379713,Won 1 Oscar. Another 7 wins & 32 nominations.,"[Action, Adventure, Thriller]",A cryptic message from the past sends James Bo...
2,tt0230575,N/A,"[Comedy, Horror]","The makers of this parody of ""Night of the Liv..."
3,tt0065988,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,"[Adventure, Comedy, Drama]",Jack Crabb is 121 years old as the film begins...
4,tt1995390,2 nominations.,"[Crime, Drama, Thriller]","When Perry and his girlfriend, Gail, cross pat..."
5,tt0066921,Nominated for 4 Oscars. Another 9 wins & 19 no...,"[Crime, Drama, Sci-Fi]","Protagonist Alex DeLarge is an ""ultraviolent"" ..."
6,tt1298650,2 wins & 31 nominations.,"[Action, Adventure, Fantasy]",Captain Jack Sparrow (Depp) crosses paths with...
7,tt0069947,Nominated for 1 Oscar. Another 1 win & 10 nomi...,"[Crime, Drama, Thriller]",It is the early 60s in France. The remaining s...
8,tt0116629,Won 1 Oscar. Another 34 wins & 34 nominations.,"[Action, Adventure, Sci-Fi]","On July 2nd, communications systems worldwide ..."
9,tt0057012,Nominated for 4 Oscars. Another 13 wins & 7 no...,"[Comedy, War]",Paranoid Brigadier General Jack D. Ripper of B...
